# Compare TLEs

In this notebook, we will demonstrate how to compare the trajectories defined by two distinct TLEs

## Setup

In [ ]:
import datetime

import numpy as np
import pandas as pd

import plotly.graph_objs as go
from plotly.subplots import make_subplots

from ostk.mathematics.objects import RealInterval

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate.spherical import AER
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE
from ostk.astrodynamics import Access
from ostk.astrodynamics.access import Generator as AccessGenerator

In [ ]:
def lla_from_state (state):
    
    lla = lla_from_position(state.get_position(), state.get_instant())

    return [
        float(lla.get_latitude().in_degrees()),
        float(lla.get_longitude().in_degrees()),
        float(lla.get_altitude().in_meters())
    ]

def lla_from_position (position, instant):
    
    return LLA.cartesian(
        position.in_frame(Frame.ITRF(), instant).get_coordinates(),
        Earth.equatorial_radius,
        Earth.flattening
    )

def position_from_lla (lla):
    
    return Position.meters(
        lla.to_cartesian(Earth.equatorial_radius, Earth.flattening),
        Frame.ITRF()
    )

def parse_ground_station_lla (ground_station):
    
    return LLA(
        Angle.degrees(ground_station['latitude']),
        Angle.degrees(ground_station['longitude']),
        Length.meters(ground_station['altitude'])
    )

def compute_aer (instant, from_position, to_position):
    
    from_lla = lla_from_position(from_position, instant)
    
    ned_frame = earth.get_frame_at(from_lla, Earth.FrameType.NED)

    from_position_NED = from_position.in_frame(ned_frame, instant)
    to_position_NED = to_position.in_frame(ned_frame, instant)

    aer = AER.from_position_to_position(from_position_NED, to_position_NED, True)
            
    return [
        float(aer.get_azimuth().in_degrees()),
        float(aer.get_elevation().in_degrees()),
        float(aer.get_range().in_meters())
    ]

def compute_time_lla_aer_state (state, ground_station_position):
    
    instant = state.get_instant()
    
    lla = lla_from_state(state)
    aer = compute_aer(instant, ground_station_position, state.get_position().in_frame(Frame.ITRF(), state.get_instant()))

    return [instant, lla[0], lla[1], lla[2], aer[0], aer[1], aer[2]]

def compute_trajectory_geometry (trajectory, interval):

    return [
        lla_from_state(state)
        for state in trajectory.get_states_at(interval.generate_grid(Duration.minutes(1.0)))
    ]

def compute_access_geometry (orbit, access, ground_station):
    
    ground_station_position = position_from_lla(parse_ground_station_lla(ground_station))

    return [
        compute_time_lla_aer_state(state, ground_station_position)
        for state in orbit.get_states_at(access.get_interval().generate_grid(Duration.seconds(1.0)))
    ]

def convert_state (instant, state):
    
    lla = LLA.cartesian(state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), Earth.equatorial_radius, Earth.flattening)
    
    return [
        repr(instant),
        float(instant.get_modified_julian_date(Scale.UTC)),
        *state.get_position().get_coordinates().transpose().tolist(),
        *state.get_velocity().get_coordinates().transpose().tolist(),
        float(lla.get_latitude().in_degrees()),
        float(lla.get_longitude().in_degrees()),
        float(lla.get_altitude().in_meters())
    ]

def generate_orbit_df (orbit, interval, step):
    
    if isinstance(interval, tuple):
        interval = Interval.closed(interval[0], interval[1]).generate_grid(step)
    
    orbit_data = [
        convert_state(instant, orbit.get_state_at(instant))
        for instant in interval
    ]

    return pd.DataFrame(
        orbit_data,
        columns = [
            '$Time^{UTC}$',
            '$MJD^{UTC}$',
            '$x_{x}^{ECI}$',
            '$x_{y}^{ECI}$',
            '$x_{z}^{ECI}$',
            '$v_{x}^{ECI}$',
            '$v_{y}^{ECI}$',
            '$v_{z}^{ECI}$',
            '$Latitude$',
            '$Longitude$',
            '$Altitude$'
        ]
    )

---

## Simulation Configuration

We define the analysis interval:

In [ ]:
start_instant = Instant.date_time(DateTime.parse('2021-06-30T20:03:55.190', DateTime.Format.ISO8601), Scale.UTC)

analysis_duration = Duration.hours(10.0)
step = Duration.minutes(1.0)

Let's bootstrap the physics environment:

In [ ]:
environment = Environment.default()
earth = environment.access_celestial_object_with_name('Earth')

In [ ]:
tle_1 = TLE(
    '1 99993U          21181.83605544  .00000000  00000-0  00000-0 0 00000',
    '2 99993 097.4983 301.8737 0009947 257.2267 135.9820 15.12223922000018'
)

orbit_1 = Orbit(SGP4(tle_1), earth)

In [ ]:
tle_2 = TLE(
    '1 70326C 21058AB  21181.83605544  .00084574  00000-0  49049-2 0    01',
    '2 70326  97.5010 301.8743 0009600 257.4138 135.7939 15.12185939    15'
)

orbit_2 = Orbit(SGP4(tle_2), earth)

Generate a time grid:

In [ ]:
instants = Interval.closed(start_instant, start_instant + analysis_duration).generate_grid(step)

Ground stations:

In [ ]:
ground_stations = {
    'Ground Station 1': {
        'identifier': 'GS1',
        'latitude': 78.23164,
        'longitude': 15.37725,
        'altitude': 483.0,
        'min_elevation_angle': 5.0,
    },
    'Ground Station 2': {
        'identifier': 'GS2',
        'latitude': -72.0021,
        'longitude': 2.5251,
        'altitude': 1401.0,
        'min_elevation_angle': 5.0,
    }
}

---

## Comparison

In [ ]:
orbit_1_df = generate_orbit_df(orbit_1, instants, step)
orbit_2_df = generate_orbit_df(orbit_2, instants, step)

Plot difference in position/velocity norms over the analysis interval.

In [ ]:
dx_ECI_norm = np.linalg.norm(
    orbit_1_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy()
    - orbit_2_df[['$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$']].to_numpy(),
    axis = 1
) / 1e3

dv_ECI_norm = np.linalg.norm(
    orbit_1_df[['$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']].to_numpy()
    - orbit_2_df[['$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']].to_numpy(),
    axis = 1
) / 1e3

print(f'Min. |Δx_ECI|: {min(dx_ECI_norm):.3f} [km]')
print(f'Avg. |Δx_ECI|: {np.mean(dx_ECI_norm):.3f} [km]')
print(f'Max. |Δx_ECI|: {max(dx_ECI_norm):.3f} [km]')
print('\n')
print(f'Min. |Δv_ECI|: {min(dv_ECI_norm):.3f} [km/s]')
print(f'Avg. |Δv_ECI|: {np.mean(dv_ECI_norm):.3f} [km/s]')
print(f'Max. |Δv_ECI|: {max(dv_ECI_norm):.3f} [km/s]')

def mean(x):
    return np.mean(x) * np.ones(np.shape(x))

data = [
    dx_ECI_norm,
    dv_ECI_norm
]

figure = go.FigureWidget(make_subplots(rows = len(data), cols = 1))

for (i, d) in enumerate(data):

    figure.append_trace(
        go.Scatter(
            y = d,
            mode = 'lines+markers',
            marker_size = 3,
            marker_symbol = 'cross',
            line_width = 1,
        ),
        row = i + 1,
        col = 1
    )

    figure.append_trace(
        go.Scatter(
            y = mean(d),
            mode = 'lines',
            line = dict(
                color = 'rgb(0, 0, 0)',
                width = 1
            ),
        ),
        row = i + 1,
        col = 1
    )

figure.update_xaxes(title_text = 'Time (min)', row = len(data), col = 1)
figure.update_yaxes(title_text = '|Δx_ECI| (km)', row = 1, col = 1)
figure.update_yaxes(title_text = '|Δv_ECI| (km/s)', row = 2, col = 1)

figure.update_layout(showlegend = False)

figure.show()

Ground Station Accesses:

In [ ]:
orbit_1_accesses = []
orbit_2_accesses = []

access_analysis_interval = Interval.closed(
    start_instant,
    start_instant + analysis_duration
)

for (ground_station_name, ground_station) in ground_stations.items():
    
    # Groud station position
    lla = parse_ground_station_lla(ground_station)
    position = position_from_lla(lla)
    
    # Access constraints
    azimuth_range = RealInterval.closed(0.0, 360.0) # [deg]
    elevation_range = RealInterval.closed(ground_station['min_elevation_angle'], 90.0) # [deg]
    range_range = RealInterval.closed(0.0, 10000e3) # [m]

    # Access generator with Azimuth-Range-Elevation constraints
    access_generator = AccessGenerator.aer_ranges(azimuth_range, elevation_range, range_range, environment)
    
    # Generate Accesses
    orbit_1_accesses += [
        (ground_station_name, ground_station, access)
        for access in access_generator.compute_accesses(access_analysis_interval, Trajectory.position(position), orbit_1)
    ]
    
    orbit_2_accesses += [
        (ground_station_name, ground_station, access)
        for access in access_generator.compute_accesses(access_analysis_interval, Trajectory.position(position), orbit_2)
    ]
    
diff_accesses_df = pd.DataFrame(
    [
        [
            ground_station_name,
            abs(orbit_1_access.get_acquisition_of_signal().get_date_time(Scale.UTC) - orbit_2_access.get_acquisition_of_signal().get_date_time(Scale.UTC)),
            abs(orbit_1_access.get_time_of_closest_approach().get_date_time(Scale.UTC) - orbit_2_access.get_time_of_closest_approach().get_date_time(Scale.UTC)),
            abs(orbit_1_access.get_loss_of_signal().get_date_time(Scale.UTC) - orbit_2_access.get_loss_of_signal().get_date_time(Scale.UTC)),
            abs(datetime.timedelta(seconds = float(orbit_1_access.get_duration().in_seconds())) - datetime.timedelta(seconds = float(orbit_2_access.get_duration().in_seconds())))
        ]
        for ((ground_station_name, ground_station, orbit_1_access), (_, _, orbit_2_access)) in zip(orbit_1_accesses, orbit_2_accesses)
    ],
    columns = [
        'Ground Station',
        '|ΔAOS|',
        '|ΔTCA|',
        '|ΔLOS|',
        '|ΔDuration|'
    ]
)

In [ ]:
diff_accesses_df

---

## Visualization

Orbit:

In [ ]:
orbit_2_df.head()

In [ ]:
access_geometry_dfs = [
    pd.DataFrame(
        compute_access_geometry(orbit_2, access, ground_station),
         columns = [
             'Time',
             'Latitude',
             'Longitude',
             'Altitude',
             'Azimuth',
             'Elevation',
             'Range'
         ]
    )
    for (_, ground_station, access) in orbit_2_accesses
]

2D plot, over **World Map**:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = orbit_2_df['$Longitude$'],
            lat = orbit_2_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 1,
                color = 'gray'
            )
        ),
        *[
            go.Scattergeo(
                lon = [ground_station['longitude']],
                lat = [ground_station['latitude']],
                text = ground_station['identifier'],
                textposition = 'middle right',
                mode = 'markers+text',
                marker = dict(
                    size = 10,
                    color = 'orange'
                )
            )
            for ground_station in ground_stations.values()
        ],
        *[
            go.Scattergeo(
                lon = access_geometry_df['Longitude'],
                lat = access_geometry_df['Latitude'],
                mode = 'lines',
                line = dict(
                    width = 2,
                    color = 'red',
                )
            )
            for access_geometry_df in access_geometry_dfs
        ]
    ],
    
    layout = go.Layout(
        title = None,
        showlegend = False,
        height=1000,
        geo = go.layout.Geo(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)'
        )
    )
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = orbit_2_df['$Longitude$'],
            lat = orbit_2_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 1,
                color = 'rgba(255, 0, 0, 0.5)'
            )
        ),
        *[
            go.Scattergeo(
                lon = [ground_station['longitude']],
                lat = [ground_station['latitude']],
                text = ground_station['identifier'],
                textposition = 'middle right',
                mode = 'markers+text',
                marker = dict(
                    size = 10,
                    color = 'orange'
                )
            )
            for ground_station in ground_stations.values()
        ],
        *[
            go.Scattergeo(
                lon = access_geometry_df['Longitude'],
                lat = access_geometry_df['Latitude'],
                mode = 'lines',
                line = dict(
                    width = 2,
                    color = 'red',
                )
            )
            for access_geometry_df in access_geometry_dfs
        ]
    ],
    layout = go.Layout(
        title = None,
        showlegend = False,
        width = 800,
        height = 800,
        geo = go.layout.Geo(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
)

figure.show()

---